## Chatbot Interativo Phi 3 Mini 4K Instruct com Whisper

### Introdução:
O Chatbot Interativo Phi 3 Mini 4K Instruct é uma ferramenta que permite aos usuários interagir com a demonstração do Microsoft Phi 3 Mini 4K Instruct usando entrada de texto ou áudio. O chatbot pode ser utilizado para diversas tarefas, como tradução, atualizações meteorológicas e coleta de informações gerais.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Crie seu Token de Acesso do Huggingface](https://huggingface.co/settings/tokens)

Crie um novo token  
Forneça um novo nome  
Selecione permissões de escrita  
Copie o token e salve-o em um local seguro


O seguinte código em Python realiza duas tarefas principais: importar o módulo `os` e configurar uma variável de ambiente.

1. Importando o módulo `os`:
   - O módulo `os` em Python fornece uma maneira de interagir com o sistema operacional. Ele permite realizar várias tarefas relacionadas ao sistema operacional, como acessar variáveis de ambiente, trabalhar com arquivos e diretórios, entre outras.
   - Neste código, o módulo `os` é importado usando a instrução `import`. Essa instrução torna a funcionalidade do módulo `os` disponível para uso no script Python atual.

2. Configurando uma variável de ambiente:
   - Uma variável de ambiente é um valor que pode ser acessado por programas que estão sendo executados no sistema operacional. É uma forma de armazenar configurações ou outras informações que podem ser usadas por vários programas.
   - Neste código, uma nova variável de ambiente está sendo configurada usando o dicionário `os.environ`. A chave do dicionário é `'HF_TOKEN'`, e o valor é atribuído a partir da variável `HUGGINGFACE_TOKEN`.
   - A variável `HUGGINGFACE_TOKEN` é definida logo acima deste trecho de código e recebe um valor de string `"hf_**************"` usando a sintaxe `#@param`. Essa sintaxe é frequentemente usada em notebooks Jupyter para permitir entrada de usuário e configuração de parâmetros diretamente na interface do notebook.
   - Ao configurar a variável de ambiente `'HF_TOKEN'`, ela pode ser acessada por outras partes do programa ou por outros programas que estão sendo executados no mesmo sistema operacional.

Resumindo, este código importa o módulo `os` e configura uma variável de ambiente chamada `'HF_TOKEN'` com o valor fornecido na variável `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Este trecho de código define uma função chamada clear_output, que é usada para limpar o output da célula atual no Jupyter Notebook ou IPython. Vamos analisar o código e entender sua funcionalidade:

A função clear_output recebe um parâmetro chamado wait, que é um valor booleano. Por padrão, wait é definido como False. Esse parâmetro determina se a função deve aguardar até que um novo output esteja disponível para substituir o output existente antes de limpá-lo.

A própria função é usada para limpar o output da célula atual. No Jupyter Notebook ou IPython, quando uma célula produz um output, como texto impresso ou gráficos, esse output é exibido abaixo da célula. A função clear_output permite que você limpe esse output.

A implementação da função não é fornecida no trecho de código, conforme indicado pela elipse (...). A elipse representa um espaço reservado para o código real que realiza a limpeza do output. A implementação da função pode envolver a interação com a API do Jupyter Notebook ou IPython para remover o output existente da célula.

De forma geral, essa função oferece uma maneira prática de limpar o output da célula atual no Jupyter Notebook ou IPython, facilitando o gerenciamento e a atualização do output exibido durante sessões interativas de codificação.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Realizar conversão de texto em fala (TTS) usando o serviço Edge TTS. Vamos analisar as implementações das funções relevantes uma por uma:

1. `calculate_rate_string(input_value)`: Esta função recebe um valor de entrada e calcula a string de velocidade para a voz TTS. O valor de entrada representa a velocidade desejada da fala, onde um valor de 1 representa a velocidade normal. A função calcula a string de velocidade subtraindo 1 do valor de entrada, multiplicando por 100 e determinando o sinal com base em se o valor de entrada é maior ou igual a 1. A função retorna a string de velocidade no formato "{sign}{rate}".

2. `make_chunks(input_text, language)`: Esta função recebe um texto de entrada e um idioma como parâmetros. Ela divide o texto de entrada em partes com base nas regras específicas do idioma. Nesta implementação, se o idioma for "English", a função divide o texto em cada ponto final (".") e remove quaisquer espaços em branco no início ou no final. Em seguida, adiciona um ponto final a cada parte e retorna a lista filtrada de partes.

3. `tts_file_name(text)`: Esta função gera um nome de arquivo para o arquivo de áudio TTS com base no texto de entrada. Ela realiza várias transformações no texto: remove um ponto final no final (se presente), converte o texto para minúsculas, remove espaços em branco no início e no final e substitui espaços por sublinhados. Em seguida, trunca o texto para um máximo de 25 caracteres (se for mais longo) ou usa o texto completo se estiver vazio. Por fim, gera uma string aleatória usando o módulo [`uuid`] e combina com o texto truncado para criar o nome do arquivo no formato "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Esta função combina vários arquivos de áudio em um único arquivo de áudio. Ela recebe uma lista de caminhos de arquivos de áudio e um caminho de saída como parâmetros. A função inicializa um objeto vazio `AudioSegment` chamado [`merged_audio`]. Em seguida, percorre cada caminho de arquivo de áudio, carrega o arquivo de áudio usando o método `AudioSegment.from_file()` da biblioteca `pydub` e adiciona o arquivo de áudio atual ao objeto [`merged_audio`]. Por fim, exporta o áudio combinado para o caminho de saída especificado no formato MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Esta função realiza a operação TTS usando o serviço Edge TTS. Ela recebe uma lista de partes de texto, a velocidade da fala, o nome da voz e o caminho de salvamento como parâmetros. Se o número de partes for maior que 1, a função cria um diretório para armazenar os arquivos de áudio individuais das partes. Em seguida, percorre cada parte, constrói um comando Edge TTS usando a função `calculate_rate_string()`, o nome da voz e o texto da parte, e executa o comando usando a função `os.system()`. Se a execução do comando for bem-sucedida, ela adiciona o caminho do arquivo de áudio gerado a uma lista. Após processar todas as partes, combina os arquivos de áudio individuais usando a função `merge_audio_files()` e salva o áudio combinado no caminho de salvamento especificado. Se houver apenas uma parte, ela gera diretamente o comando Edge TTS e salva o áudio no caminho de salvamento. Por fim, retorna o caminho de salvamento do arquivo de áudio gerado.

6. `random_audio_name_generate()`: Esta função gera um nome de arquivo de áudio aleatório usando o módulo [`uuid`]. Ela gera um UUID aleatório, converte para uma string, pega os primeiros 8 caracteres, adiciona a extensão ".mp3" e retorna o nome de arquivo de áudio aleatório.

7. `talk(input_text)`: Esta função é o ponto de entrada principal para realizar a operação TTS. Ela recebe um texto de entrada como parâmetro. Primeiro, verifica o comprimento do texto de entrada para determinar se é uma frase longa (maior ou igual a 600 caracteres). Com base no comprimento e no valor da variável `translate_text_flag`, determina o idioma e gera a lista de partes de texto usando a função `make_chunks()`. Em seguida, gera um caminho de salvamento para o arquivo de áudio usando a função `random_audio_name_generate()`. Por fim, chama a função `edge_free_tts()` para realizar a operação TTS e retorna o caminho de salvamento do arquivo de áudio gerado.

No geral, essas funções trabalham juntas para dividir o texto de entrada em partes, gerar um nome de arquivo para o arquivo de áudio, realizar a operação TTS usando o serviço Edge TTS e combinar os arquivos de áudio individuais em um único arquivo de áudio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

A implementação de duas funções: convert_to_text e run_text_prompt, assim como a declaração de duas classes: str e Audio.

A função convert_to_text recebe um audio_path como entrada e transcreve o áudio para texto usando um modelo chamado whisper_model. A função primeiro verifica se a flag gpu está configurada como True. Se estiver, o whisper_model é utilizado com certos parâmetros, como word_timestamps=True, fp16=True, language='English' e task='translate'. Se a flag gpu estiver configurada como False, o whisper_model é usado com fp16=False. A transcrição resultante é então salva em um arquivo chamado 'scan.txt' e retornada como texto.

A função run_text_prompt recebe uma mensagem e um chat_history como entrada. Ela utiliza a função phi_demo para gerar uma resposta de um chatbot com base na mensagem de entrada. A resposta gerada é então passada para a função talk, que converte a resposta em um arquivo de áudio e retorna o caminho do arquivo. A classe Audio é usada para exibir e reproduzir o arquivo de áudio. O áudio é exibido usando a função display do módulo IPython.display, e o objeto Audio é criado com o parâmetro autoplay=True, para que o áudio comece a tocar automaticamente. O chat_history é atualizado com a mensagem de entrada e a resposta gerada, e uma string vazia e o chat_history atualizado são retornados.

A classe str é uma classe embutida no Python que representa uma sequência de caracteres. Ela fornece vários métodos para manipular e trabalhar com strings, como capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill e mais. Esses métodos permitem realizar operações como busca, substituição, formatação e manipulação de strings.

A classe Audio é uma classe personalizada que representa um objeto de áudio. Ela é usada para criar um player de áudio no ambiente do Jupyter Notebook. A classe aceita vários parâmetros, como data, filename, url, embed, rate, autoplay e normalize. O parâmetro data pode ser um array numpy, uma lista de amostras, uma string representando um nome de arquivo ou URL, ou dados PCM brutos. O parâmetro filename é usado para especificar um arquivo local de onde carregar os dados de áudio, e o parâmetro url é usado para especificar um URL para baixar os dados de áudio. O parâmetro embed determina se os dados de áudio devem ser incorporados usando um URI de dados ou referenciados da fonte original. O parâmetro rate especifica a taxa de amostragem dos dados de áudio. O parâmetro autoplay determina se o áudio deve começar a tocar automaticamente. O parâmetro normalize especifica se os dados de áudio devem ser normalizados (reajustados) para o alcance máximo possível. A classe Audio também fornece métodos como reload para recarregar os dados de áudio a partir de um arquivo ou URL, e atributos como src_attr, autoplay_attr e element_id_attr para recuperar os atributos correspondentes para o elemento de áudio em HTML.

No geral, essas funções e classes são usadas para transcrever áudio para texto, gerar respostas em áudio de um chatbot e exibir e reproduzir áudio no ambiente do Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
